In [12]:
import numpy as np
from collections import OrderedDict as odict
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm

This notebook computes the latencies and bandwidths of the three primitive function types

In [13]:
#(hardware name, number of nodes)
filesD = {
         'knl_mpi1':('knl',1), 'knl_mpi2':('knl',2), 'knl_mpi4':('knl',4),
         'skl_mpi1':('skl',1), 'skl_mpi2':('skl',2), 'skl_mpi4':('skl',4),
         'i5':('i5',1),
         'p100_mpi1':('p100',1), 'p100_mpi2':('p100',2), 'p100_mpi4':('p100',4),
         'v100_mpi1':('v100',1), 'v100_mpi2':('v100',2), 'v100_mpi4':('v100',4),
         'gtx1060':('gtx1060',1)
        }

files = odict(sorted(filesD.items(), key= lambda t : t[1][1]))
pd.set_option('precision',1)

#### Axpby and Dot Latencies
The latencies are determined by taking the minimum of the average runtimes 

#### Axpby and Dot Bandwidths
The bandwidths are determined by taking the average bandwidth of the 30 bandwidths corresponding to the 3 largest sizes.

#### Dx-Dy Latencies
As in Axpby 

#### Dx-Dy Bandwidths
Since the efficiency of the matrix-vector multiplications depends on the polynomial coefficient we should compute these bandwidths separately


In [25]:
names={'axpby':3,'dot':2,'dx':3, 'dy':3}
#ns=[3,4]
values = []
for f, v in files.items() :#{'knl_mpi2':('knl',2)}.items():
    runtimes=pd.read_csv('benchmark_'+f+'.csv', delimiter=' ')
    #add size and bandwidth columns
    runtimes.insert(0,'size', 8*runtimes['n']*runtimes['n']*runtimes['Nx']*runtimes['Ny']/1e6/v[1]) #inplace transformation
    for name,memops in names.items() :
        runtimes.insert(0,name+'_bw',runtimes['size']/1000*memops/runtimes[name])
    runtimes = runtimes.assign( dxdy=(runtimes['dx']+runtimes['dy'])/2)
    runtimes = runtimes.assign( dxdy_bw=2.0*runtimes['dx_bw']*runtimes['dy_bw']/(runtimes['dx_bw']+runtimes['dy_bw']))
    #compute one version with aggregated grouped sizes and one without
    avgruntimes=runtimes.groupby(['n', 'Nx','Ny','size']).agg(['mean', 'std'])
    avgruntimes=avgruntimes.reset_index(level=['n','Nx','Ny','size'])
    avgruntimes.sort_values(by='size',inplace=True) #sort by size
    runtimes.sort_values(by='size',inplace=True)
    ##first compute axpby and dot latencies and bandwidths 
    nmax = 3
    s =30

    line = []
    l=len(runtimes)
    line.append(v[0]) #0
    line.append(v[1]) #1
    line.append(runtimes[l-s:l]['axpby_bw'].mean()) #2
    line.append(runtimes[l-s:l]['axpby_bw'].std())  #3
    line.append(avgruntimes[0:nmax][('axpby','mean')].mean()/1e-6) #4
    line.append(avgruntimes[0:nmax][('axpby','mean')].min()/1e-6)  #5

    line.append( line[5] - avgruntimes['size'][0]*names['axpby']/line[2]/1e-3) #6
    if line[6] <0 : line[6] = 0 

    line.append(runtimes[l-s:l]['dot_bw'].mean()) #7
    line.append(runtimes[l-s:l]['dot_bw'].std())  #8 
    line.append(avgruntimes[0:nmax][('dot','mean')].mean()/1e-6) #9
    line.append(avgruntimes[0:nmax][('dot','mean')].min()/1e-6)  #10
    line.append(line[10] - avgruntimes['size'][0]*names['dot']/line[2]/1e-3) #11
    if line[11] <0 : line[11] = 0 
    ##now compute latency and bandwidths of dx and y
 
    for n in [2,3,4,5]:
        #take n
        dxdy=runtimes[runtimes['n']==n]
        
        avgdxdy = avgruntimes[avgruntimes['n']==n]
        dxdy=dxdy.sort_values(by='size')
        avgdxdy=avgdxdy.sort_values(by='size') #sort by size

        Nx = 767 # compute among the four greates sizes
        line.append(dxdy[dxdy['Nx']>=Nx]['dxdy_bw'].mean())
        line.append(dxdy[dxdy['Nx']>=Nx]['dxdy_bw'].std())   
        line.append(avgdxdy[0:nmax][('dxdy','mean')].mean()/1e-6)
        line.append(avgdxdy[('dxdy','mean')].min()/1e-6)
        line.append(avgdxdy[('dxdy','mean')].min()/1e-6 - avgdxdy['size'].loc[avgdxdy[('dxdy','mean')].idxmin()]*names['dx']/line[2]/1e-3)
        if line[len(line)-1] <0 : line[len(line)-1] = 0        
    #print(line)    
    values.append(line)

In [26]:
#now construct new table with values from previous cell      
tuples=[('arch','',''),('nodes','','')]        
for q in ['axpby','dot','dxdy2','dxdy3','dxdy4','dxdy5']:
    tuples.append((q,'bw','avg'))
    tuples.append((q,'bw','std'))
    tuples.append((q,'lat','avg'))
    tuples.append((q,'lat','min'))
    tuples.append((q,'lat','bw'))

cols=pd.MultiIndex.from_tuples(tuples)
arr = pd.DataFrame(values,index=files.keys(), columns=cols)
arr.sort_values(by='arch',inplace=True)
arr.set_index(['arch','nodes'],inplace=True)
#arr.loc[:,[('dot','bw','avg'),('dot','lat','avg')]]
arr

axpby                                 dot                 \
                   bw           lat                    bw            lat   
                  avg    std    avg    min     bw     avg    std     avg   
arch    nodes                                                              
gtx1060 1      157.05   0.06  23.19   3.51   0.00   26.50   0.10  199.90   
i5      1       29.99   0.19  30.58  12.38   0.00    9.31   0.04  316.67   
knl     1      442.58  11.47  15.44  13.40   9.38  143.57   8.43   80.22   
        2      449.58  14.75  28.49  14.69  12.71  131.60   4.83  103.79   
        4      426.24  39.05  26.00  19.00  17.95  113.32   7.15  133.96   
p100    1      552.83   1.02   4.15   3.09   0.25  345.78   1.91   61.09   
        2      554.17   0.11   3.32   3.02   1.60  339.78   2.92   53.48   
        4      554.67   0.89   3.18   3.13   2.42  324.29   8.39   54.05   
skl     1      205.00  10.77   7.96   6.44   0.00  196.55   6.82   32.71   
        2      210.19  18.68   8.55   6.40   2.66  181.83  12.79   36.77   
        4      228.55  22.70   7.48   5.97   4.25  174.61   4.53   41.89   
v100    1      845.61   0.92   3.65   3.07   1.21  593.90   1.64   36.64   
        2      846.95   0.43   3.34   3.17   2.24  581.65   5.85   38.34   
        4      844.76   1.83   3.32   3.27   2.80  552.12  10.26   39.96   

                                ...     dxdy4                                  \
                                ...        bw             lat                   
                  min      bw   ...       avg    std      avg     min      bw   
arch    nodes                   ...                                             
gtx1060 1      131.63  124.95   ...     79.99  15.63   322.56   72.50   32.44   
i5      1      117.43   82.46   ...     25.78   1.00  1212.12  208.32    0.00   
knl     1       68.68   66.01   ...    118.66   8.74   225.94   58.43   42.38   
        2       96.53   95.22   ...    107.39   4.38   303.24  212.71  181.11   
        4      132.05  131.36   ...     88.01  12.43   288.84  216.77  200.11   
p100    1       56.38   54.49   ...    200.74   1.70   148.55   30.15   18.77   
        2       49.60   48.65   ...    190.13   7.69   128.09   66.59   60.92   
        4       48.67   48.20   ...    166.38  18.98   131.54   91.55   88.71   
skl     1       26.50   21.39   ...    114.62   9.82   375.41   85.71   55.02   
        2       30.80   28.30   ...    105.53  15.38   161.20   76.00   61.03   
        4       40.29   39.14   ...     95.20  16.49   102.64   63.66   56.77   
v100    1       35.72   34.48   ...    668.46   8.94    47.13   12.10    4.66   
        2       35.97   35.35   ...    576.35  50.67    72.64   52.91   49.19   
        4       37.40   37.09   ...    442.27  94.38   101.58   88.95   87.09   

                dxdy5                                  
                   bw             lat                  
                  avg    std      avg     min      bw  
arch    nodes                                          
gtx1060 1       61.06  18.77   571.43  125.05   62.45  
i5      1       20.76   1.41  1956.57  340.41   12.57  
knl     1       95.87   3.13   460.34   92.70   67.63  
        2       88.36   4.74   480.17  342.23  329.89  
        4       75.17   6.85   382.07  291.45  265.42  
p100    1      162.61  18.08   273.05   62.74   44.96  
        2      166.11   3.92   191.42   79.98   71.11  
        4      153.52   9.66   166.79  107.20  102.77  
skl     1       95.77   9.62   425.80  130.23   82.28  
        2      101.53  10.12   249.22   99.68   76.30  
        4       94.21  17.76   152.38   73.54   62.79  
v100    1      536.92  68.45    85.47   23.49   11.86  
        2      525.85  27.11    94.06   58.51   52.70  
        4      428.65  70.49   112.68   91.62   88.71  

[14 rows x 30 columns]

In [27]:
#arr=arr.reset_index()

In [28]:
#define conversion function 
def toString(x): 
    np.ceil(x)
    #string = '%.1f'% x
    string = '%d' %np.ceil(x)
    return string

In [29]:
addto = []
for n in ['axpby','dot','dxdy2','dxdy3','dxdy4','dxdy5']:
    arr.loc[:,(n,'bw','string')]= arr[n]['bw']['avg'].apply(toString) +" ± "+arr[n]['bw']['std'].apply(toString)
    addto.append((n,'lat','bw'))
    addto.append((n,'bw','string'))

#make a table for display
nicetable=arr[addto]
drop = nicetable.columns.droplevel(2)
nicetable.columns=drop
#nicetable.reset_index(inplace=True)
#nicetable.set_index('arch')
newindex=[('i5',1)]
for n in ['skl','knl']:
    for m in [1,2,4]:
        newindex.append((n,m))
newindex.append(('gtx1060',1))
for n in ['p100','v100']:
    for m in [1,2,4]:
        newindex.append((n,m))
    
nicetable=nicetable.reindex(newindex)
nicetable

axpby               dot             dxdy2              dxdy3  \
                 lat        bw     lat        bw     lat         bw     lat   
arch    nodes                                                                 
i5      1       0.00    30 ± 1   82.46    10 ± 1    0.00     28 ± 3    0.00   
skl     1       0.00  205 ± 11   21.39   197 ± 7   27.35   152 ± 40   37.31   
        2       2.66  211 ± 19   28.30  182 ± 13   39.41   133 ± 42   48.73   
        4       4.25  229 ± 23   39.14   175 ± 5   39.23   114 ± 17   48.31   
knl     1       9.38  443 ± 12   66.01   144 ± 9   19.18   249 ± 12   25.24   
        2      12.71  450 ± 15   95.22   132 ± 5   86.83   154 ± 33  123.65   
        4      17.95  427 ± 40  131.36   114 ± 8  137.84   102 ± 27  169.14   
gtx1060 1       0.00   158 ± 1  124.95    27 ± 1    1.72    131 ± 1   11.95   
p100    1       0.25   553 ± 2   54.49   346 ± 2    4.54    288 ± 3    9.26   
        2       1.60   555 ± 1   48.65   340 ± 3   47.91   232 ± 33   50.52   
        4       2.42   555 ± 1   48.20   325 ± 9   83.78   161 ± 49   84.23   
v100    1       1.21   846 ± 1   34.48   594 ± 2    3.12   803 ± 17    3.63   
        2       2.24   847 ± 1   35.35   582 ± 6   49.88  514 ± 136   49.14   
        4       2.80   845 ± 2   37.09  553 ± 11   84.97  287 ± 129   85.12   

                           dxdy4             dxdy5            
                      bw     lat        bw     lat        bw  
arch    nodes                                                 
i5      1         29 ± 1    0.00    26 ± 1   12.57    21 ± 2  
skl     1        153 ± 6   55.02  115 ± 10   82.28   96 ± 10  
        2       134 ± 28   61.03  106 ± 16   76.30  102 ± 11  
        4       132 ± 30   56.77   96 ± 17   62.79   95 ± 18  
knl     1       170 ± 23   42.38   119 ± 9   67.63    96 ± 4  
        2       125 ± 10  181.11   108 ± 5  329.89    89 ± 5  
        4       103 ± 25  200.11   89 ± 13  265.42    76 ± 7  
gtx1060 1        111 ± 1   32.44   80 ± 16   62.45   62 ± 19  
p100    1        240 ± 2   18.77   201 ± 2   44.96  163 ± 19  
        2       216 ± 15   60.92   191 ± 8   71.11   167 ± 4  
        4       177 ± 31   88.71  167 ± 19  102.77  154 ± 10  
v100    1       732 ± 12    4.66   669 ± 9   11.86  537 ± 69  
        2       580 ± 86   49.19  577 ± 51   52.70  526 ± 28  
        4      396 ± 127   87.09  443 ± 95   88.71  429 ± 71

#### Assumptions
- there are three basic functions: trivially parallel(axpby), nearest neighbor (dxdy), global reduction (dot)
- each can be represented by the single node bandwidth, the single node latency and the multinode latency

#### But
- does not capture cache effect e.g. in SKl

In [30]:
index = ['i5','skl','knl','gtx1060','p100','v100']  
lines = []
for arch in  index: 
    line = []
    line.append(arch)
    #first the bandwidths
    line.append( arr.loc[(arch,1),('axpby','bw','avg')] )
    for n in ['dot','dxdy2','dxdy3','dxdy4','dxdy5']:
        line.append( arr.loc[(arch,1),(n,'bw','avg')] /line[1])
    for n in ['axpby','dot','dxdy2'] :
        line.append( arr.loc[(arch,1),(n,'lat','bw')] )
        if arch == 'i5' or arch == 'gtx1060':
            line.append(None)
        else:
            line.append( arr.loc[(arch,4),(n,'lat','bw')] )
    lines.append(line)
    
tuples=['arch']     

for n in ['axpby','dot','dxdy2','dxdy3','dxdy4','dxdy5']:
    tuples.append(n+'_bw')
for n in ['axpby','dot','dxdy']:
    tuples.append(n+'_lat_shared')
    tuples.append(n+'_lat_dist')
cols=tuples
toDisk = pd.DataFrame(lines, columns=cols)
toDisk.to_csv('performance.csv',sep=' ',index=False)

In [31]:
pd.set_option('precision',2)
test = pd.read_csv('performance.csv',delimiter=' ')
test

,arch,axpby_bw,dot_bw,dxdy2_bw,dxdy3_bw,dxdy4_bw,dxdy5_bw,axpby_lat_shared,axpby_lat_dist,dot_lat_shared,dot_lat_dist,dxdy_lat_shared,dxdy_lat_dist
0,i5,29.99,0.31,0.93,0.97,0.86,0.69,0.00,NaN,82.46,NaN,0.00,NaN
1,skl,205.00,0.96,0.74,0.74,0.56,0.47,0.00,4.25,21.39,39.14,27.35,39.23
2,knl,442.58,0.32,0.56,0.38,0.27,0.22,9.38,17.95,66.01,131.36,19.18,137.84
3,gtx1060,157.05,0.17,0.83,0.70,0.51,0.39,0.00,NaN,124.95,NaN,1.72,NaN
4,p100,552.83,0.63,0.52,0.43,0.36,0.29,0.25,2.42,54.49,48.20,4.54,83.78
5,v100,845.61,0.70,0.95,0.86,0.79,0.63,1.21,2.80,34.48,37.09,3.12,84.97
